In [2]:
from dask.distributed import Client

client = Client(memory_limit='4GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 14.90 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:44777,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 14.90 GiB
Comm: tcp://127.0.0.1:45445,Total threads: 2
Dashboard: http://127.0.0.1:41707/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:46757,


In [3]:
from scripts import model
from minsearch import minsearch
import dask.dataframe as dd

ddf = dd.read_csv("../datasets/ministers/New/Apostle Femi Lazarus.csv_new.csv")
ddf = ddf.drop(columns=["Unnamed: 0"])
ddf

/home/cirec/.cache/pypoetry/virtualenvs/cirec-01-llm-falcon-KwmbxNdi-py3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


,minister,channel_name,title,description,transcript
npartitions=1,,,,,
,string,string,string,string,string
,...,...,...,...,...


In [4]:
n_partitions = 10

ddf = ddf.repartition(npartitions=n_partitions)
ddf = ddf.repartition(partition_size="100MB")

ddf

,minister,channel_name,title,description,transcript
npartitions=1,,,,,
,string,string,string,string,string
,...,...,...,...,...


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size= 1000, chunk_overlap = 200)
def chunking(text):
    return text_splitter.split_text(text)


meta = ddf.head(0)
page_chunks = {}

for i, text in enumerate(ddf["transcript"], start=1):
    chunk = text_splitter.split_text(text)
    page_chunks[i] = chunk   


In [6]:
standard_data = []
for key, val in page_chunks.items():
    for i, v in enumerate(val, start = 1):
        data = {
            "top_level" : str(key),
            "id" : str(i),
            "chunk_text" : v
        }
        standard_data.append(data)

In [7]:
import dask.bag as db

data_as_bag = db.from_sequence(standard_data, npartitions=293)
data_as_bag.repartition(partition_size="50MB")
data_as_bag.take(2)

({'top_level': '1',
  'id': '1',
  'chunk_text': "welcome to an encounter with the spirit, word and prayer through the prolific Apostolic and Teaching Ministry of apostle. am Lazarus, lead pastor, sperm church Global. it is his vision to raise God's end time on. God has not called you to prove you are the best. he has not as a leader, you are a broker of gift and talent. so brace up for an experience. father. we receive insight for this conversation today in the name of Jesus that you will speak to us by yourself. in Jesus, mighty name, now let's turn our Bibles to the Book of Genesis, chapter number 12. Genesis, chapter number 12. we'll start the reading from verse one, Genesis 12, from verse one. now the Lord had said to Abraham: get thee out of your country and from your kindred and from your father's house onto the land, I will show you, he said, I will make you and I will make you, or make of you, a great nation and I will bless you and make your name great and you shall be a bles

In [8]:
from minsearch import Index

class BasicSearch:
    
    def __init__(self, data, num_results = 5):
        self.data = data
        self.num_results = num_results
        self.index = None
        self.search_kwargs = {}


    def create_index(self, **kwargs):
        '''
        data: pd.DataFrame
        return: Index
        
        This function takes a DataFrame and returns an Index object.
        
        '''
        print(kwargs)
        
        text_fields = kwargs["text_fields"]
        keyword_fields = kwargs["keyword_fields"]
        
        self.index = Index(
            text_fields = text_fields,
            keyword_fields = keyword_fields
        )
        
        self.index.fit(self.data)    
        
    def basic_search(self, query) -> list:
        '''
        query: str
        return: str
        
        This function takes a query and retx12  o9urns the same query.
        
        '''
        # filter_dict = {"category": "Questions about God"}
        boost_dict = {"question": 3, "answer": 1, "sub.category": 1}
        return self.index.search(query, boost_dict, num_results=self.num_results)

In [ ]:
search = BasicSearch(standard_data, 30)
search.create_index(
    text_fields = ["chunk_text"], 
    keyword_fields=["top_level"]
)

{'text_fields': ['chunk_text'], 'keyword_fields': ['top_level']}


In [10]:
# print(results)

In [14]:
from scripts.model import llm, build_prompt
query = "dominion"
results = search.basic_search(query)

print(results)
# You're also given a list of transcripts whose \
#         texts are not properly correct or right, your goal is to answer the QUESTION \
#             in a very subtle way based on the provided CONTEXT \
#     about christian spirituality like faith, hope, love, \
#         spiritual virtues and Life as a believer of this spirituality.
#     Answer the QUESTION as if you are a spiritual assistant and a guide using the \
#         CONTEXT given. Don't provide personal opinions or beliefs.
#     If the CONTEXT doesn't match or contain the answer, \
#         give NONE as the response.
    
prompt_template = """
    Generate a sermon based on the provided context. According to the topic given in the query
    CONTEXT : {context}
    
    QUESTION : {query}
"""

context = ""
for doc in results:
    context = f'{results}chunk_text: {doc["chunk_text"]} question: {query}'
prompt_template = prompt_template.format(context=context, query = query)

llm(prompt_template)

[{'top_level': '190', 'id': '2', 'chunk_text': "are you with me? it matters. one of the major secret of walking in the Supernatural: as powerful as importation is I, I, I think it is not even importation as such. that's one of the major secret of walking in the realm of consistent Dominion. there's a secret to it. when Minister AB, you know, mounted The Altar and was ministering, I, I laughed in. my heart cut, I saw it. there's something called awareness to be conscious. that's where we all Minister from right. there are things you can't take from me. I don't care how dead and atmosphere is. I'm aware when I show up it will tilt because God will show up. you see, it is an awareness, there is a measure of Grace that will not flow through you if you are not aware. when Peter met the man at the gate, called beautiful and he said: silver and gold we don't have, but such as I have said, say such as I have. you see, I, I understand we want to see far, but sometimes don't don't see fire yet l

'## The Dominion Within: A Sermon on Inherent Power\n\nBrothers and sisters, have you ever felt inadequate? Like you’re lacking something essential to live the victorious life God has promised?  Have you felt the pressure of the world, the whispers of doubt, telling you that you’re not enough? This morning, we’re going to talk about dominion, a word that resonates with power, authority, and control. But more importantly, we’re going to uncover the dominion that already resides *within* you.\n\nOur text today draws inspiration from Psalm 114, a powerful declaration of God’s dominion and the inheritance of His people.  "When Israel went out of Egypt… Judah was his sanctuary and Israel his dominion." The sea fled, Jordan was driven back, mountains skipped like rams! This wasn’t through their own strength, but through the presence of God *with* them. This speaks to a fundamental truth: our dominion is rooted in our relationship with the Almighty.\n\nWe also heard today about awareness, abo